In [ ]:
import os
os.chdir('../')
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
exp_combination = [
    # Default Case
    {"Dataset": "CIFAR10", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Global"},
    {"Dataset": "SVHN", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Global"},
    {"Dataset": "PATHMNIST", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Global"},
    {"Dataset": "ORGANAMNIST", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Global"},
    {"Dataset": "DERMAMNIST", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Global"},
    {"Dataset": "CIFAR10", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Local Only"},
    {"Dataset": "SVHN", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Local Only"}, 
    {"Dataset": "PATHMNIST", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Local Only"},    
    {"Dataset": "ORGANAMNIST", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Local Only"},
    {"Dataset": "DERMAMNIST", "Model": "CNN4Conv", "Dir": "0.1", "Buget Size": "5%", "Model Init": "Random", "Query Selector": "Local Only"},
]

In [ ]:
df = pd.read_csv("./final_results.csv", dtype=str)

In [ ]:
algorithms = ["LoGo", 'Random', "Entropy", "CoreSet", 'Badge', "GCNAL", "ALFA-Mix" ]

In [ ]:
comparison_matrix = pd.DataFrame(np.zeros((len(algorithms), len(algorithms))))
comparison_matrix.columns = algorithms
comparison_matrix.index = algorithms
# comparison_matrix["Random"]["Badge"]

In [ ]:
seed_cnt = 4

for exp in exp_combination:
    print(exp)
    temp_df = df
    
    for k, v in exp.items():
        temp_df = temp_df[temp_df[k] == v]

    per_algorithm = list(itertools.permutations(algorithms, 2))
    
    for algo1 in algorithms:
        for algo2 in algorithms:
            # print("-"* 20)
            # print(algo1, algo2)
            defeat_cnt = 0

            algo1_lst = []
            algo2_lst = []

            for seed in range(1, seed_cnt + 1):
                # Algorithm 1 acc
                acc_lst = [float(acc) for acc in temp_df[(temp_df['Seed']==str(seed)) & (temp_df['Algorithm']==algo1)]['Acc'].values[0].split()]
                algo1_lst.append(acc_lst)

                # Algorithm 2 acc
                acc_lst = [float(acc) for acc in temp_df[(temp_df['Seed']==str(seed)) & (temp_df['Algorithm']==algo2)]['Acc'].values[0].split()]
                algo2_lst.append(acc_lst)

            num_label_set = len(acc_lst)

            for i in range(1, num_label_set):
                algo1_v_lst = [temp_lst[i] for temp_lst in algo1_lst]
                algo2_v_lst = [temp_lst[i] for temp_lst in algo2_lst]

                mean = np.mean([a1_v - a2_v for a1_v, a2_v in zip(algo1_v_lst, algo2_v_lst)])
                std = np.sqrt(sum([(a1_v - a2_v - mean)**2 for a1_v, a2_v in zip(algo1_v_lst, algo2_v_lst)]) / (seed_cnt-1))

                t_v = np.sqrt(seed_cnt) * mean / std

                if t_v > 2.776:
                    defeat_cnt += 1

            # print(algo1, algo2, defeat_cnt / num_label_set)
            comparison_matrix[algo1][algo2] += defeat_cnt / num_label_set
            
            # print(defeat_cnt / num_label_set)


In [ ]:
comparison_matrix

In [ ]:
ax = sns.heatmap(comparison_matrix.T, annot=True,  cmap="viridis")
plt.tick_params(axis='both', which='major', labelsize=10, labelbottom = False, bottom=False, top = False, labeltop=True)
plt.yticks(rotation=0)
plt.xticks(rotation=45)
plt.show()